<!--
tags: ["RAG"]
description: |
    Explore the use of AutoGen's RagAgent for tasks like code generation from docstrings, answering complex questions with human feedback, and exploiting features like Update Context, custom prompts, and few-shot learning.
-->

# Using RagAgent for Retrieval-Augmented Code Generation and Question Answering

Introducing an agent capable of performing Retrieval-Augmented Generation (RAG) for the given message.

Upon receipt of a message, the agent employs RAG to generate a reply. It retrieves documents based on the message, then generates a reply using both the retrieved documents and the message itself. Additionally, it supports automatic context updates during the conversation, either autonomously or at the user`s request.

We also support enabling the RAG capability for any conversable agent with `Ragability`.

## Table of Contents
We'll demonstrate six examples of using RetrieveChat for code generation and question answering:

- [Example 1: Use RagAgent to help generate code](#example-1)
- [Example 2: Use RagAgent to answer a multi-hop question.](#example-2)
- [Example 3: Enable RAG capability for any conversable agent.](#example-3)

````{=mdx}
:::info Requirements
Some extra dependencies are needed for this notebook, which can be installed via pip:

```bash
pip install pyautogen[rag] flaml[automl]
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [1]:
import os
import sys
import logging
import autogen
from autogen.agentchat.contrib.rag import RagAgent, logger
from autogen.agentchat.contrib.rag.splitter import SUPPORTED_EXTENSIONS
from autogen.agentchat.contrib.capabilities.rag_capability import Ragability

logger.setLevel(logging.DEBUG)

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

/home/lijiang1/anaconda3/envs/autogen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


models to use:  ['gpt-35-turbo', 'gpt-35-turbo-0613']


configuration).
:::
````

## Construct agents for RAG

We start by initializing the `RagAgent` and `UserProxyAgent`. The `RagAgent` will be responsible for replying to `UserProxyAgent` with retrieval-augmented generation based on the given knowledge base contained in the `docs_path`.

In [2]:
print(f"Accepted file formats for `docs_path`: {SUPPORTED_EXTENSIONS}")

Accepted file formats for `docs_path`: ['.txt', '.json', '.csv', '.tsv', '.md', '.html', '.htm', '.rtf', '.rst', '.jsonl', '.log', '.xml', '.yaml', '.yml', '.pdf']


In [3]:
llm_config = {
    "timeout": 60,
    "config_list": config_list,
}


def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


userproxy = autogen.UserProxyAgent(
    name="userproxy",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    code_execution_config=False,  # {"use_docker": False, "work_dir": ".tmp"},
    # default_auto_reply="Reply `TERMINATE` if the task is done.",
    description="The boss who ask questions and give tasks.",
)

# Check the docstring of RagAgent for more details on the configuration.
rag_config = {
    "docs_path": [
        "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
        "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md",
        os.path.join(os.path.abspath(""), "..", "website", "docs"),
    ],
    "overwrite": True,  # set to False to avoid overwriting the existing documents
}

rag = RagAgent(
    name="rag",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    llm_config=llm_config,
    rag_config=rag_config,
    code_execution_config=False,
    description="Assistant that can answer questions and generate code based on retrieved documents.",
)

/home/lijiang1/anaconda3/envs/autogen/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
2024-02-26 08:00:23,441 - utils.py:   16 - DEBUG - Processing file: ./tmp/download/Integrate%20-%20Spark.md, url: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md
2024-02-26 08:00:23,799 - utils.py:   16 - DEBUG - Split ./tmp/download/Integrate%20-%20Spark.md into 2 chunks.
2024-02-26 08:00:23,800 - utils.py:   16 - DEBUG - Processing file: ./tmp/download/Research.md, url: https:/

### Example 1

[Back to top](#table-of-contents)

Use RagAgent to help generate sample code and automatically run the code and fix errors if there is any.

Problem: How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.

In [4]:
code_problem = "How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached."
rag.rag_filter_document = {"$contains": "spark"}  # filter documents that contain "spark"
_ = userproxy.initiate_chat(rag, message=code_problem)

userproxy (to rag):

How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.

--------------------------------------------------------------------------------


2024-02-26 08:00:26,572 - utils.py:   16 - DEBUG - is_code_execution_result: False
2024-02-26 08:00:26,573 - utils.py:   16 - DEBUG - Input message: How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.
2024-02-26 08:00:26,573 - utils.py:   16 - DEBUG - Received raw message: How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.
2024-02-26 08:00:26,574 - utils.py:   16 - DEBUG - Performing RAG for message: How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.
2024-02-26 08:00:26,603 - utils.py:   16 - DEBUG - Task predicted: `code`
2024-02-26 08:00:26,603 - utils.py:   16 - DEBUG - __call__ took 0.03 seconds.
2024-02-26 08:00:26,604 - utils.py:   16 - DEBUG - Refined message for d

rag (to userproxy):

You can use FLAML for a classification task and train the model in 30 seconds while using Spark to parallelize the training. Force cancel jobs if the time limit is reached using the following code snippet:

```python
import pandas as pd
from pyspark.sql import SparkSession
from flaml import AutoML

spark = SparkSession.builder \
    .appName("FLAML") \
    .config("spark.sql.execution.arrow.enabled","true") \
    .getOrCreate()

train_data = spark.read.format("csv").option("header", "true").option(
    "inferSchema", "true").load("train.csv")

train_pd = train_data.toPandas()
X_train = train_pd.drop(columns=['target'])
y_train = train_pd['target']

automl = AutoML()

settings = {
    "time_budget": 30,
    "metric": 'accuracy',
    "task": 'classification',
    "n_jobs": -1,
    "model_history": True,
    "log_file_name": "log.txt",
    "estimator_list": ['lgbm', 'rf', 'xgboost'],
    "use_spark": True,
    "force_cancel": True,
    "num_samples": -1,
    "seed": 4

2024-02-26 08:00:26,823 - utils.py:   16 - DEBUG - History message: {'content': 'How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.', 'role': 'user'}
2024-02-26 08:00:26,823 - utils.py:   16 - DEBUG - History message: {'content': 'You can use FLAML for a classification task and train the model in 30 seconds while using Spark to parallelize the training. Force cancel jobs if the time limit is reached using the following code snippet:\n\n```python\nimport pandas as pd\nfrom pyspark.sql import SparkSession\nfrom flaml import AutoML\n\nspark = SparkSession.builder \\\n    .appName("FLAML") \\\n    .config("spark.sql.execution.arrow.enabled","true") \\\n    .getOrCreate()\n\ntrain_data = spark.read.format("csv").option("header", "true").option(\n    "inferSchema", "true").load("train.csv")\n\ntrain_pd = train_data.toPandas()\nX_train = train_pd.drop(columns=[\'target\'])\ny_train = tra

### Example 2

[Back to top](#table-of-contents)

Use RagAgent to answer a multi-hop question.

Problem: The common authors of the paper FLAML and paper AutoGen.

In [5]:
# reset the assistant. Always reset the assistant before starting a new conversation.
rag.reset()

qa_problem = "The common authors of the paper FLAML and paper AutoGen."
rag.rag_filter_document = None  # clear filter
_ = userproxy.initiate_chat(rag, message=qa_problem)

userproxy (to rag):

The common authors of the paper FLAML and paper AutoGen.

--------------------------------------------------------------------------------


2024-02-26 08:00:26,835 - utils.py:   16 - DEBUG - is_code_execution_result: False
2024-02-26 08:00:26,835 - utils.py:   16 - DEBUG - Input message: The common authors of the paper FLAML and paper AutoGen.
2024-02-26 08:00:26,836 - utils.py:   16 - DEBUG - Received raw message: The common authors of the paper FLAML and paper AutoGen.
2024-02-26 08:00:26,836 - utils.py:   16 - DEBUG - Performing RAG for message: The common authors of the paper FLAML and paper AutoGen.
2024-02-26 08:00:26,864 - utils.py:   16 - DEBUG - Task predicted: multihop
2024-02-26 08:00:26,865 - utils.py:   16 - DEBUG - __call__ took 0.03 seconds.
2024-02-26 08:00:26,865 - utils.py:   16 - DEBUG - Refined message for db query: ['Who are the authors of the paper FLAML', 'Who are the authors of the paper AutoGen', 'Are there any authors common to both papers']
2024-02-26 08:00:26,941 - utils.py:   16 - DEBUG - retrieve_docs took 0.08 seconds.
2024-02-26 08:00:26,944 - utils.py:   16 - DEBUG - get_docs_by_ids took 0.

rag (to userproxy):

The common authors of the paper FLAML and paper AutoGen are Qingyun Wu, Erkang Zhu, and Chi Wang.

Source: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /datadrive/autogen/notebook/../website/docs/Research.md /datadrive/autogen/notebook/../website/docs/Ecosystem.md

--------------------------------------------------------------------------------
userproxy (to rag):



--------------------------------------------------------------------------------


2024-02-26 08:00:26,995 - utils.py:   16 - DEBUG - History message: {'content': 'The common authors of the paper FLAML and paper AutoGen.', 'role': 'user'}
2024-02-26 08:00:26,995 - utils.py:   16 - DEBUG - History message: {'content': 'The common authors of the paper FLAML and paper AutoGen are Qingyun Wu, Erkang Zhu, and Chi Wang.\n\nSource: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /datadrive/autogen/notebook/../website/docs/Research.md /datadrive/autogen/notebook/../website/docs/Ecosystem.md', 'role': 'assistant'}
2024-02-26 08:00:26,996 - utils.py:   16 - DEBUG - is_code_execution_result: False
2024-02-26 08:00:26,997 - utils.py:   16 - DEBUG - Input message: 
2024-02-26 08:00:26,997 - utils.py:   16 - DEBUG - Received raw message: The common authors of the paper FLAML and paper AutoGen.
2024-02-26 08:00:26,995 - utils.py:   16 - DEBUG - History message: {'content': 'The common authors of the paper FLAML and paper AutoGen are Qingyun Wu, Erkan

### Example 3

[Back to top](#table-of-contents)

Enable RAG capability for any conversable agent.

First, let's try to answer the same question in [Example 2](#example-2) with a default `AssistantAgent`. It couldn't answer it well because AutoGen paper is not in the LLM model's training data.

In [6]:
normal_assistant = autogen.AssistantAgent(name="normal assistant", llm_config=llm_config, max_consecutive_auto_reply=3)
message = "The common authors of the paper FLAML and paper AutoGen."

In [7]:
normal_assistant.reset()

_ = userproxy.initiate_chat(normal_assistant, message=message)

userproxy (to normal assistant):

The common authors of the paper FLAML and paper AutoGen.

--------------------------------------------------------------------------------
The common authors of the paper FLAML and paper AutoGen.

--------------------------------------------------------------------------------
normal assistant (to userproxy):

Plan:
1. Search for the full names of the authors of paper FLAML and paper AutoGen.
2. Collect the common authors of the two papers.

Code:
```python
# Search for the full names of the authors of paper FLAML and paper AutoGen.
import requests
from bs4 import BeautifulSoup

def search_authors(paper_title):
    base_url = "https://scholar.google.com/"
    headers = {'User-Agent': 'Mozilla/5.0'}
    query = "q={}".format(paper_title)
    
    url = base_url + "scholar?" + query
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    author_tags = soup.select('.gs_a')
    authors = []
    for tag i

Second, let's enable `Ragability` for the same assistant agent. As you can see, with the `Ragability` the agent can now answer the question correctly.

In [8]:
rag_config["overwrite"] = False
ragability = Ragability(llm_config=llm_config, rag_config=rag_config, verbose=2)
ragability.add_to_agent(normal_assistant)

normal_assistant.reset()

_ = userproxy.initiate_chat(normal_assistant, message=message)

2024-02-26 08:00:27,348 - utils.py:   19 - INFO - Use the existing collection `autogen-rag`.
2024-02-26 08:00:27,350 - utils.py:   16 - DEBUG - insert_docs took 0.00 seconds.


userproxy (to normal assistant):

The common authors of the paper FLAML and paper AutoGen.

--------------------------------------------------------------------------------
normal assistant (to rag_agent):

The common authors of the paper FLAML and paper AutoGen.

--------------------------------------------------------------------------------


2024-02-26 08:00:27,378 - utils.py:   16 - DEBUG - is_code_execution_result: False
2024-02-26 08:00:27,379 - utils.py:   16 - DEBUG - Input message: The common authors of the paper FLAML and paper AutoGen.
2024-02-26 08:00:27,379 - utils.py:   16 - DEBUG - Received raw message: The common authors of the paper FLAML and paper AutoGen.
2024-02-26 08:00:27,379 - utils.py:   16 - DEBUG - Performing RAG for message: The common authors of the paper FLAML and paper AutoGen.
2024-02-26 08:00:27,407 - utils.py:   16 - DEBUG - Task predicted: multihop
2024-02-26 08:00:27,408 - utils.py:   16 - DEBUG - __call__ took 0.03 seconds.
2024-02-26 08:00:27,408 - utils.py:   16 - DEBUG - Refined message for db query: ['Who are the authors of the paper FLAML', 'Who are the authors of the paper AutoGen', 'Are there any authors common to both papers']
2024-02-26 08:00:27,490 - utils.py:   16 - DEBUG - retrieve_docs took 0.08 seconds.
2024-02-26 08:00:27,492 - utils.py:   16 - DEBUG - get_docs_by_ids took 0.

rag_agent (to normal assistant):

The common authors of the paper FLAML and paper AutoGen are Qingyun Wu, Erkang Zhu, and Chi Wang.

Source: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /datadrive/autogen/notebook/../website/docs/Research.md /datadrive/autogen/notebook/../website/docs/Ecosystem.md

--------------------------------------------------------------------------------
normal assistant (to userproxy):

The common authors of the paper FLAML and paper AutoGen are Qingyun Wu, Erkang Zhu, and Chi Wang. Here are the links to their research pages:
- https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md
- /datadrive/autogen/notebook/../website/docs/Research.md
- /datadrive/autogen/notebook/../website/docs/Ecosystem.md

Is there anything else I can help you with?

--------------------------------------------------------------------------------
userproxy (to normal assistant):



----------------------------------------------